In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from data_cleaner import *
from room_energy import energyLossAllRooms, energy_to_building

In [2]:
df_fcu_sth_raw = pd.read_csv('data/Gnd floor FCU Sth 16032020.csv')
df_fcu_nth_raw = pd.read_csv('data/Gnd floor FCU North 16032020.csv')
df_ahu_raw = pd.read_csv('data/Gnd AHU multi list 16032020.csv')
df_chiller_boiler_raw = pd.read_csv('data/more_Data/chillers boilers thermal Feb 23032020.csv')
room_info_raw = pd.read_csv('data/Room Details.csv')
df_ltb_space_raw = pd.read_csv('data/LTB Space Data.csv')

In [ ]:
df_ltb_temps = create_temp_df(df_fcu_sth_raw, df_fcu_nth_raw, df_ahu_raw)
df_chiller_boiler_power = create_chiller_boiler_power_df(df_chiller_boiler_raw)
df_room_info = create_room_info_df(room_info_raw)

# Ignore rooms with no external walls
df_ltb_temps = df_ltb_temps.drop(['AHU-B1-02 ZnTmp_1', 'AHU-B1-02 ZnTmp_2','FCU-16 ZnTmp', 'FCU-15 ZnTmp'], axis=1)
df_room_info = df_room_info[df_room_info['AHU / FCU'] != 'AHU-B1-02']
df_room_info = df_room_info[df_room_info['AHU / FCU'] != 'FCU-16']
df_room_info = df_room_info[df_room_info['AHU / FCU'] != 'FCU-15']


In [ ]:
df_energy_room_change = energyLossAllRooms(df_ltb_temps, df_room_info, time_frame=["06:00", "18:00"])
df_energy_external = energy_to_building(df_ltb_temps, df_room_info, time_frame=["06:00", "18:00"])

# Plotting results

#### Single Day Temperature results

In [ ]:
START_DATE = '2020-02-06 01:10:0'  
END_DATE = '2020-02-06 23:59:59'
df_ltb_temps_day = create_temp_df(df_fcu_sth_raw, df_fcu_nth_raw, df_ahu_raw, START_DATE, END_DATE)

time_frame = ["00:00", "23:30"]
zn_tmp = 'FCU-04 ZnTmp'
sa_tmp = 'FCU-04 SaTmp'
df_ltb_temps_day = df_ltb_temps_day.between_time(time_frame[0], time_frame[1])
delta_temp = df_ltb_temps_day['OaTmp'] - df_ltb_temps_day[zn_tmp]
 
# Get current axis
fig= plt.figure(figsize=(20,10))
ax = plt.gca()

df_ltb_temps_day['OaTmp'].plot(kind='line', ax=ax, legend=True)
df_ltb_temps_day[zn_tmp].plot(kind='line', ax=ax, legend=True)
df_ltb_temps_day[sa_tmp].plot(kind='line', ax=ax, legend=True)
df_ltb_temps_day['FCU-24 ZnTmp'].plot(kind='line', ax=ax, legend=True)


#delta_temp.plot(kind='line', ax=ax, legend=True)

plt.show()

#### Full Time Period

In [ ]:
time_frame = ["06:00", "18:00"]
df_ltb_temps_open = df_ltb_temps.between_time(time_frame[0], time_frame[1])
 
# Get current axis
fig= plt.figure(figsize=(20,10))
ax = plt.gca()

for col in df_ltb_temps_open.columns:
    if 'Sa' not in col and 'OaRH' not in col:
        df_ltb_temps_day[col].plot(kind='line', ax=ax, legend=True)

plt.show()

In [ ]:
 # Get current axis
fig= plt.figure(figsize=(20,10))
ax = plt.gca()
ax.set_title("Room Energy Change over February Period")
ax.set_ylabel("Energy due Temperature Change (kJ)")
for col in df_energy_room_change.columns:
    df_energy_room_change[col].plot(kind='line', ax=ax, legend=True)

plt.legend(loc="lower right")
plt.show()

In [ ]:
 # Get current axis
fig= plt.figure(figsize=(20,10))
ax = plt.gca()
ax.set_title("Energy Exchange with Outside over February Period")
ax.set_ylabel("Energy Change Externally (kJ)")
for col in df_energy_external.columns:
    df_energy_external[col].plot(kind='line', ax=ax, legend=True)
    
plt.legend(loc="lower right")
plt.show()

Assuming change in room energy = incoming energy from outside - reduction of energy from AC

-> reduction of energy from AC = incoming - change in room 

In [ ]:
df_effectice_ac_energy = df_energy_external - df_energy_room_change
df_effectice_ac_energy.iloc[1:3]



In [ ]:
df_energy_external.iloc[1:3]

In [ ]:
df_energy_room_change.iloc[1:3]

In [ ]:
 # Get current axis
fig= plt.figure(figsize=(20,10))
ax = plt.gca()
ax.set_title("Energy Used ")
ax.set_ylabel("Energy Change Externally (kJ)")
for col in df_effectice_ac_energy.columns:
    df_effectice_ac_energy[col].plot(kind='line', ax=ax, legend=True)
    
plt.legend(loc="lower right")
plt.show()

In [ ]:
ac_energy_work = df_effectice_ac_energy.abs().sum().sum()

In [ ]:
chiller_energy, boiler_energy = get_power_used(df_chiller_boiler_power)
total_ac_energy_used = chiller_energy + boiler_energy

ac_floor_area = df_room_info['Total Area'].sum()
ltb_foor_area = df_ltb_space_raw['Room Area'].sum()
gnd_floor_area = df_ltb_space_raw[df_ltb_space_raw['Floor'] == '01']['Room Area'].sum()

ac_area_multiplier = ltb_foor_area / ac_floor_area
building_ac_energy_work = ac_area_multiplier * ac_energy_work


In [ ]:
BTU = building_ac_energy_work/ 1.055
total_ac_wh = total_ac_energy_used/ 3.6  # kJ to Wh

In [ ]:
SEER = BTU/total_ac_wh

In [ ]:
SEER